In [832]:
import numpy as np
from sklearn.neighbors import NearestNeighbors, RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix,classification_report , precision_recall_fscore_support
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from statistics import mean
from tqdm import tqdm
import open3d as o3d

In [833]:
pts_file_path = "Vaihingen3D_Traininig.pts"
points = np.loadtxt(pts_file_path, skiprows=1)
features=points[:,:3]
labels= points[:, 6].astype(int) #6

In [864]:
import plotly.graph_objects as go

coordinates = points[:,:3]
labelsv =points[:, 6] # 3 /6 
rgb_values = np.zeros((len(coordinates ), 3)) 

label_colors = {
    0: [255, 255, 0],   # Label 0: powerline
    1: [128, 179, 0],   # Label 1: low vegetation
    2: [153, 179, 179], # Label 2: surface
    3: [230, 153, 0.0], # Label 3: car
    4: [153, 0, 0],     # Label 4: fence
    5: [0.0, 0.0, 255], # Label 5: roof
    6: [230, 77, 255],   # Label 6: facade 
    7: [153, 255, 102],   # Label 7: shurb
    8: [0.0, 255, 0.0],   # Label 8: tree
}

# Assign colors to each label based on the color mapping dictionary
rgb_values = np.zeros((len(coordinates ), 3))
for label in label_colors:
    mask = labelsv == label
    rgb_values[mask] = label_colors[label]

# Create trace for each label
traces = []
for label in label_colors:
    mask = labelsv == label
    trace = go.Scatter3d(
        x=coordinates[mask, 0],
        y=coordinates[mask, 1],
        z=coordinates[mask, 2],
        mode='markers',
        marker=dict(
            size=1.5,
            color='rgb({},{},{})'.format(*label_colors[label]),
            opacity=0.8
        ),
        name=f"Label {label}"
    )
    traces.append(trace)

# Create layout
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

# Create figure
fig = go.Figure(data=traces, layout=layout)
fig.update_layout(autosize=True) # remove height=800
fig.show(renderer="browser") 

# Curvature Calculation

In [835]:
rnn = NearestNeighbors(n_neighbors=None,radius=3.0)
rnn.fit(features)
distances, indices = rnn.radius_neighbors(features)

In [836]:
rnn_features = []

def feature_extraction(data, indices):
   
    for i in tqdm(range(len(data))):      
        if(indices[i].shape==(2,) or indices[i].shape==(1,) or indices[i].shape==(0,)):            
            surface_variations=0.31     
        else:
                   
            neighborhood = data[indices[i]]  
        
            pca = PCA(n_components=3)
            pca.fit(neighborhood)
            eigenvalues = pca.explained_variance_
        
                # calculate eigenvalue-based features
            surface_variations = eigenvalues[2] / np.sum(eigenvalues)
        
            rnn_features.append([data[i,0],data[i,1],data[i,2], labels[i],surface_variations])


    return np.array(rnn_features)

curvaturefeatures = feature_extraction(features, indices)
curvature=curvaturefeatures[:,4].reshape(-1, 1) 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27371/27371 [00:07<00:00, 3732.13it/s]


## Curvature : Kmeans

In [837]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2,init='k-means++', max_iter=300, random_state=42)
kmeans.fit(curvature)

KMeans(n_clusters=2, random_state=42)

In [838]:
cur1=kmeans.cluster_centers_[0]
cur2=kmeans.cluster_centers_[1]
print(cur1)
print(cur2)

[0.01508624]
[0.13425612]


In [839]:
import collections
collections.Counter(kmeans.labels_)

Counter({1: 6677, 0: 20691})

In [840]:
ct= #calculate Threshold 

[0.04416024]


## Divide :Regular and Scatter

In [841]:
regular_region=[]
scatter_region=[]
#ct=0.02197497 #test
#ct=0.06297905  #train
for i in range(len(curvaturefeatures)):
    if curvaturefeatures[i,4] < ct:
        regular_region.append([curvaturefeatures[i,0],curvaturefeatures[i,1],curvaturefeatures[i,2],curvaturefeatures[i,3]])
    else:
        scatter_region.append([curvaturefeatures[i,0],curvaturefeatures[i,1],curvaturefeatures[i,2],curvaturefeatures[i,3]])

In [842]:
regular_region=np.array(regular_region)
scatter_region=np.array(scatter_region)

# Verticality Calculation (Plannar & Vertical Region)

In [843]:
point_cloud = regular_region[:,:3]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=3.0, max_nn=30))#  search_param=o3d.geometry.KDTreeSearchParamRadius(radius=3.0)
normals = np.asarray(pcd.normals)

In [844]:
point_cloud = regular_region[:,:3]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=3.0, max_nn=30))

# Get normal vectors
normals = np.asarray(pcd.normals)

# Calculate verticality for each point
verticality = 1 - np.abs(normals[:, 2])  # Assuming Z-axis is vertical
verticality=verticality.reshape(-1, 1) 
print("Verticality values for the first few points:")
print(verticality[:3])

Verticality values for the first few points:
[[5.82662057e-03]
 [1.82694416e-01]
 [1.52054407e-04]]


## Verticality : Kmeans

In [845]:
kmeans = KMeans(n_clusters=2,init='k-means++', max_iter=100, random_state=42)
kmeans.fit(verticality)

KMeans(max_iter=100, n_clusters=2, random_state=42)

In [846]:
v1=kmeans.cluster_centers_[0]
v2=kmeans.cluster_centers_[1]
print(v1)
print(v2)

[0.03701609]
[0.91048654]


In [847]:
collections.Counter(kmeans.labels_)

Counter({0: 17561, 1: 325})

In [848]:
vt= #calculate Threshold

[0.05288761]


## Divide : Planar and Vertical Region

In [849]:
planar_region=[]
vertical_region=[]
vt=0.22692663 #train
for i in range(len(regular_region)):  
    if verticality[i]<vt:
        planar_region.append([regular_region[i,0],regular_region[i,1],regular_region[i,2],regular_region[i,3]])
    else:
        vertical_region.append([regular_region[i,0],regular_region[i,1],regular_region[i,2],regular_region[i,3]])

In [850]:
vertical_region=np.array(vertical_region)
planar_region=np.array(planar_region)

# Omnivariance (low and high omnivariance region)

In [851]:
rnn = NearestNeighbors(n_neighbors=None,radius=3.0)
rnn.fit(scatter_region[:,0:3])
distances, indices = rnn.radius_neighbors(scatter_region[:,0:3])

In [852]:
omnivariance_features = []

def feature_extraction(data, indices):
   
    for i in tqdm(range(len(data))):   
        if(indices[i].shape==(2,) or indices[i].shape==(1,) or indices[i].shape==(0,)):            
            omnivariance=2.1
        else:            
            
            neighborhood = data[indices[i]]  
        
            pca = PCA(n_components=3)
            pca.fit(neighborhood)
            eigenvalues = pca.explained_variance_
        
                # calculate eigenvalue-based features
            omnivariance = (eigenvalues[0] + eigenvalues[1]+ eigenvalues[2]) ** (1/3)
        
            omnivariance_features.append([data[i,0],data[i,1],data[i,2], scatter_region[i,3],omnivariance])


    return np.array(omnivariance_features)
    
omnivarianceFeatures = feature_extraction(scatter_region[:,0:3], indices)
omnivariance=omnivarianceFeatures[:,4].reshape(-1, 1) 

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9482/9482 [00:02<00:00, 3696.26it/s]


## Omnivariance -Kmeans

In [853]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2,init='k-means++', max_iter=100, random_state=42)
kmeans.fit(omnivariance)

KMeans(max_iter=100, n_clusters=2, random_state=42)

In [854]:
om1=kmeans.cluster_centers_[0]
om2=kmeans.cluster_centers_[1]
print(om1)
print(om2)

[1.61850228]
[1.37811637]


In [855]:
collections.Counter(kmeans.labels_)

Counter({1: 2884, 0: 6594})

In [856]:
ot=#calculate threshold

[1.54535678]


In [858]:
omnivarianceFeatures[:,4]

array([1.35147479, 1.34421019, 1.26035644, ..., 1.35256138, 1.37476906,
       1.33716348])

## Divide : Low & High Omnivariance Region

In [859]:
low_omni_region=[]
high_omni_region=[]

for i in range(len(omnivarianceFeatures)):  
    if omnivarianceFeatures[i,4] < ot:
        low_omni_region.append([omnivarianceFeatures[i,0],omnivarianceFeatures[i,1],omnivarianceFeatures[i,2],omnivarianceFeatures[i,3]])
    else:
        high_omni_region.append([omnivarianceFeatures[i,0],omnivarianceFeatures[i,1],omnivarianceFeatures[i,2],omnivarianceFeatures[i,3]])

In [860]:
low_omni_region=np.array(low_omni_region)
high_omni_region=np.array(high_omni_region

# Entropy-based Neighborhood 

# Adaptive neighborhood - Normal Direction

# Simplicial based neighborhood

# Combined

In [681]:
point_cloud_region = np.vstack((vertical_region,planar_region, low_omni_region,high_omni_region))

In [682]:
geofeatures=  np.vstack((planar_features,vertical_features,low_omni_features,high_omni_features))

# Visualization

In [790]:
import plotly.graph_objects as go

coordinates = planar_region[:,:3]
coolabels =planar_region[:, 3].astype(int)  # Assuming the label is in the fourth column and converting to integers
rgb_values = np.zeros((len(planar_region[:,:3]), 3)) 

label_colors = {
    0: [1, 1, 0],   # Label 0: powerline
    1: [0.5, 0.7, 0],   # Label 1: low vegetation
    2: [153, 179, 179], # Label 2: surface
    3: [0.9, 0.6, 0.0], # Label 3: car
    4: [0.6, 0, 0],     # Label 4: fence
    5: [0.0, 0.0, 255], # Label 5: roof
    6: [0.9, 0.3, 1],   # Label 6: facade 
    7: [153, 255, 102],   # Label 7: shurb
    8: [0.0, 255, 0.0],   # Label 8: tree
}

# Assign colors to each label based on the color mapping dictionary
rgb_values = np.zeros((len(planar_region[:,:3]), 3))
for label in label_colors:
    mask = coolabels == label
    rgb_values[mask] = label_colors[label]

# Create trace for each label
traces = []
for label in label_colors:
    mask = coolabels == label
    trace = go.Scatter3d(
        x=coordinates[mask, 0],
        y=coordinates[mask, 1],
        z=coordinates[mask, 2],
        mode='markers',
        marker=dict(
            size=1,
            color='rgb({},{},{})'.format(*label_colors[label]),
            opacity=0.8
        ),
        name=f"Label {label}"
    )
    traces.append(trace)

# Create layout
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

# Create figure
fig = go.Figure(data=traces, layout=layout)

# Show plot
fig.show()


In [707]:
import plotly.graph_objects as go

coordinates = vertical_region[:,:3]
coolabels =vertical_region[:,3].astype(int)   # Assuming the label is in the fourth column and converting to integers
rgb_values = np.zeros((len(vertical_region[:,:3]), 3)) 

label_colors = {
    0: [255, 255, 0],   # Label 0: powerline
    1: [128, 179, 0],   # Label 1: low vegetation
    2: [255,255,255], # Label 2: surface[153, 179, 179],
    3: [230, 153, 0.0], # Label 3: car
    4: [153, 0, 0],     # Label 4: fence
    5: [0.0, 0.0, 255], # Label 5: roof
    6: [230, 77, 255],   # Label 6: facade 
    7: [153, 255, 102],   # Label 7: shurb
    8: [0.0, 255, 0.0],   # Label 8: tree
}

# Assign colors to each label based on the color mapping dictionary
rgb_values = np.zeros((len(vertical_region[:,:3]), 3))
for label in label_colors:
    mask = coolabels  == label
    rgb_values[mask] = label_colors[label]

# Create trace for each label
traces = []
for label in label_colors:
    mask = coolabels  == label
    trace = go.Scatter3d(
        x=coordinates[mask, 0],
        y=coordinates[mask, 1],
        z=coordinates[mask, 2],
        mode='markers',
        marker=dict(
            size=1,
            color='rgb({},{},{})'.format(*label_colors[label]),
            opacity=0.8
        ),
        name=f"Label {label}"
    )
    traces.append(trace)

# Create layout
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

# Create figure
fig = go.Figure(data=traces, layout=layout)

# Show plot
fig.show()


In [708]:
import plotly.graph_objects as go

coordinates = high_omni_region[:,:3]
coolabels = high_omni_region[:, 3].astype(int) # Assuming the label is in the fourth column and converting to integers
rgb_values = np.zeros((len(coordinates ), 3)) 

label_colors = {
    0: [255, 255, 0],   # Label 0: powerline
    1: [128, 179, 0],   # Label 1: low vegetation
    2: [153, 179, 179], # Label 2: surface
    3: [230, 153, 0.0], # Label 3: car
    4: [153, 0, 0],     # Label 4: fence
    5: [0.0, 0.0, 255], # Label 5: roof
    6: [230, 77, 255],   # Label 6: facade 
    7: [153, 255, 102],   # Label 7: shurb
    8: [0.0, 255, 0.0],   # Label 8: tree
}

# Assign colors to each label based on the color mapping dictionary
rgb_values = np.zeros((len(coordinates), 3))
for label in label_colors:
    mask = coolabels == label
    rgb_values[mask] = label_colors[label]

# Create trace for each label
traces = []
for label in label_colors:
    mask = coolabels == label
    trace = go.Scatter3d(
        x=coordinates[mask, 0],
        y=coordinates[mask, 1],
        z=coordinates[mask, 2],
        mode='markers',
        marker=dict(
            size=1.5,
            color='rgb({},{},{})'.format(*label_colors[label]),
            opacity=0.8
        ),
        name=f"Label {label}"
    )
    traces.append(trace)

# Create layout
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

# Create figure
fig = go.Figure(data=traces, layout=layout)

# Show plot
fig.show()


In [709]:
import plotly.graph_objects as go

coordinates = low_omni_region[:,:3]
coolabels = low_omni_region[:, 3].astype(int) # Assuming the label is in the fourth column and converting to integers
rgb_values = np.zeros((len(coordinates ), 3)) 

label_colors = {
    0: [255, 255, 0],   # Label 0: powerline
    1: [128, 179, 0],   # Label 1: low vegetation
    2: [153, 179, 179], # Label 2: surface
    3: [230, 153, 0.0], # Label 3: car
    4: [153, 0, 0],     # Label 4: fence
    5: [0.0, 0.0, 255], # Label 5: roof
    6: [230, 77, 255],   # Label 6: facade 
    7: [153, 255, 102],   # Label 7: shurb
    8: [0.0, 255, 0.0],   # Label 8: tree
}

# Assign colors to each label based on the color mapping dictionary
rgb_values = np.zeros((len(coordinates), 3))
for label in label_colors:
    mask = coolabels == label
    rgb_values[mask] = label_colors[label]

# Create trace for each label
traces = []
for label in label_colors:
    mask = coolabels == label
    trace = go.Scatter3d(
        x=coordinates[mask, 0],
        y=coordinates[mask, 1],
        z=coordinates[mask, 2],
        mode='markers',
        marker=dict(
            size=1.5,
            color='rgb({},{},{})'.format(*label_colors[label]),
            opacity=0.8
        ),
        name=f"Label {label}"
    )
    traces.append(trace)

# Create layout
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

# Create figure
fig = go.Figure(data=traces, layout=layout)

# Show plot
fig.show()


# Classification

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(geofeatures[:,3:16],geofeatures[:,0], test_size=0.3, random_state=42)
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)
rf_classifier.fit(X_train, y_train)

In [ ]:
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(precision_recall_fscore_support(y_test, y_pred, average='macro'))

In [ ]:
y_pred = rf_classifier.predict(geofeatures[:,3:16])
accuracy = accuracy_score(geofeatures[:,0], y_pred)
print("Accuracy:", accuracy)
print(precision_recall_fscore_support(geofeatures[:,0], y_pred, average='macro'))

In [ ]:
import seaborn as sns

# Define the class labels
class_labels = ['Low Vegetation', 'Impervious Surface', 'Car', 'Roof', 'Facade', 'Shrub', 'Tree']

conf_matrix = confusion_matrix(y_test, y_pred)

# Normalize the confusion matrix
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Plotting the normalized confusion matrix with a color legend
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_normalized, annot=True, fmt='.2f', cmap='YlGnBu', cbar=True, cbar_kws={'label': 'Normalized Count'}, xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Normalized Confusion Matrix')
plt.show()

In [ ]:
valLabes_color = { 
    0: [1,1,0],   # Label 0: powerline
    1: [0.5, 0.7, 0],   # Label 1: low vegetation
    2: [0.6, 0.7,.7], # Label 2: surface
    3: [0.9, 0.6, 0.0],# Label 3: car
    4: [0.6,0,0], #label 4: fence
    5: [0.0, 0.0, 1.0], #label 5 : roof
    6: [0.9,0.3,1], # facade 
    7: [0.6,1,0.4],#shurb
    8: [0.0,1,0.0]
}
fig, axs = plt.subplots(1, 2, figsize=(20,10))

for label, color in valLabes_color.items():
    mask = y_pred == label
    axs[0].scatter(point_cloud_region [mask, 0],point_cloud_region [mask, 1], color=color, s=0.5, label=y_pred)
axs[0].set_axis_off()

axs[1].scatter(point_cloud_region  [:, 0], point_cloud_region [:, 1], c=geofeatures[:,0]-y_pred, cmap= "Reds",s=0.5*(geofeatures[:,0]-y_pred))

#cmap = plt.cm.rainbow,
axs[1].set_axis_off()
plt.show()